# Multiple Linear Regression

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD

from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model

### 1. data

In [ ]:
# Raw Data Loading
data = r'/content/drive/MyDrive/멋사_AI/MLDL/data/ozone.csv'
df_oz = pd.read_csv(data)

#
# 데이터 추출
training_data = df_oz[["Solar.R", "Wind", "Temp", "Ozone"]]

# 결측치 제거
training_data = training_data.dropna()

# 이상치 데이터
zscore_threshold = 1.8

display(training_data[np.abs(stats.zscore(training_data["Ozone"])) > zscore_threshold])

# 이상치 제거 데이터
training_data = training_data[np.abs(stats.zscore(training_data["Ozone"])) < zscore_threshold]
display(training_data)

x_data = training_data.drop('Ozone', axis=1).values
t_data = training_data["Ozone"].values.reshape(-1, 1)

# 정규화 진행
scaled_x = MinMaxScaler()  # 독립변수용 scaling
scaled_t = MinMaxScaler()  # 종속변수용 scaling

scaled_x.fit(x_data)  # 최대-최소값 확인
scaled_t.fit(t_data)

x_data_norm = scaled_x.transform(x_data)  # 값을 변경
t_data_norm = scaled_t.transform(t_data)

,Solar.R,Wind,Temp,Ozone
29,223.0,5.7,79,115.0
61,269.0,4.1,84,135.0
85,223.0,8.0,85,108.0
98,255.0,4.0,89,122.0
100,207.0,8.0,90,110.0
116,238.0,3.4,81,168.0
120,225.0,2.3,94,118.0


,Solar.R,Wind,Temp,Ozone
0,190.0,7.4,67,41.0
1,118.0,8.0,72,36.0
2,149.0,12.6,74,12.0
3,313.0,11.5,62,18.0
6,299.0,8.6,65,23.0
...,...,...,...,...
147,20.0,16.6,63,14.0
148,193.0,6.9,70,30.0
150,191.0,14.3,75,14.0
151,131.0,8.0,76,18.0


### 2. Tensorflow keras

In [ ]:
# Model 생성
keras_model = Sequential()

# Flatten은 input layer의 역할을 수행
# Flatten 안의 node들은 feature의 개수만큼 필요 => 3
keras_model.add(Flatten(input_shape=(3,)))

# Dense는 output layer의 역할을 수행
output_layer = Dense(units=1, activation="linear")
keras_model.add(output_layer)

keras_model.compile(optimizer=SGD(learning_rate=1e-2), loss="mse")

keras_model.fit(x_data_norm, t_data_norm, epochs=1000, verbose=0)

In [ ]:
# Modeld을 이용한 예측
my_data = np.array([[150.0, 10.0, 80.0]])  # 모델에는 2차원으로 데이터 입력, 데이터는 실수로
my_data_scaled = scaled_x.transform(my_data)

result_scaled = keras_model.predict(my_data_scaled)
result = scaled_t.inverse_transform(result_scaled)

print(f"결과로 알아낸 오존량은: {result}")  # [[40.598953]]

1/1 [==============================] - 0s 19ms/step
결과로 알아낸 오존량은: [[40.598953]]


### 3. sklearn

In [ ]:
# Model 생성
sklearn_model = linear_model.LinearRegression()

# Model 학습(정규화하지 않은 데이터를 이용)
sklearn_model.fit(x_data, t_data)

# Model로 예측
my_data = np.array([[150.0, 10.0, 80.0]])
result = sklearn_model.predict(my_data)

print(result)  # [[40.34973385]], sklearn 정규화를 하지 않아도 편리하게 비슷한 값 출력

[[40.34973385]]
